In [4]:
"""
Aurora Bay RAG System
"""
import vertexai
import pandas as pd
from google.cloud import bigquery
from vertexai.generative_models import GenerativeModel
import google.auth
import os

# ============================================================================
# Configuration
# ============================================================================
PROJECT_ID = "qwiklabs-gcp-00-cc0593714b16"
LOCATION = "us-central1"
DATASET_ID = "aurora_bay"
TABLE_ID = "faqs"
GENERATIVE_MODEL = "gemini-2.5-flash"

# ============================================================================
# SQL Sanitization Function - ADD THIS HERE
# ============================================================================

def sanitize_sql_input(text):
    """Sanitize user input for SQL queries to prevent syntax errors"""
    return text.replace("'", "''")

# Initialize clients
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = bigquery.Client(project=PROJECT_ID)

print("🚀 Aurora Bay RAG System Initializing...")
print(f"📍 Project: {PROJECT_ID}")
print(f"📍 Location: {LOCATION}")
print(f"🤖 Generative Model: {GENERATIVE_MODEL}")

# ============================================================================
# Step 1: Check Table Structure and Map Columns
# ============================================================================

def analyze_table_structure():
    """Analyze the actual table structure and map columns"""
    try:
        table_ref = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
        table = client.get_table(table_ref)

        print("✅ Table structure detected:")
        for field in table.schema:
            print(f"   - {field.name}: {field.field_type}")

        # Let's sample some data to understand the structure
        sample_query = f"SELECT * FROM `{table_ref}` LIMIT 3"
        sample_data = client.query(sample_query).to_dataframe()

        print("\n📊 Sample data preview:")
        print(sample_data)

        # Based on typical FAQ structure, map columns
        # string_field_0 is likely questions, string_field_1 is answers
        return {
            'question_col': 'string_field_0',
            'answer_col': 'string_field_1'
        }

    except Exception as e:
        print(f"❌ Table analysis failed: {e}")
        return None

# ============================================================================
# Step 2: Fixed Semantic Search with Correct Column Names AND SANITIZATION
# ============================================================================

def semantic_search_fixed(user_question, top_k=3):
    """Semantic search using correct column names with SQL sanitization"""
    print("🔍 Searching FAQs...")

    # SANITIZE THE USER INPUT - ADD THIS LINE
    sanitized_question = sanitize_sql_input(user_question)

    try:
        # Fixed SQL with correct column names AND SANITIZED INPUT
        search_sql = f"""
        WITH scored_faqs AS (
          SELECT
            string_field_0 as question,
            string_field_1 as answer,
            -- Enhanced multi-factor relevance scoring
            CASE
              WHEN LOWER(string_field_0) LIKE LOWER('%{sanitized_question}%') THEN 0.95
              WHEN LOWER(string_field_1) LIKE LOWER('%{sanitized_question}%') THEN 0.85
              WHEN LOWER(CONCAT(string_field_0, ' ', string_field_1)) LIKE LOWER('%{sanitized_question}%') THEN 0.80
              WHEN REGEXP_CONTAINS(LOWER(string_field_0), r'(?i)\\b({'|'.join(sanitized_question.lower().split())})\\b') THEN 0.75
              WHEN REGEXP_CONTAINS(LOWER(string_field_1), r'(?i)\\b({'|'.join(sanitized_question.lower().split())})\\b') THEN 0.65
              ELSE 0.1
            END as relevance_score
          FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
        )
        SELECT
          question,
          answer,
          relevance_score,
          (1 - relevance_score) as distance
        FROM scored_faqs
        WHERE relevance_score > 0.5  -- Higher threshold to filter out weak matches
        ORDER BY relevance_score DESC
        LIMIT {top_k}
        """

        results = client.query(search_sql).to_dataframe()

        if not results.empty:
            print(f"✅ Found {len(results)} relevant FAQs")
            for i, (_, row) in enumerate(results.iterrows(), 1):
                print(f"   {i}. {row['question'][:60]}... (score: {row['relevance_score']:.3f})")
        else:
            print("❌ No highly relevant FAQs found")

        return results

    except Exception as e:
        print(f"❌ Search error: {e}")
        return pd.DataFrame()

# ============================================================================
# Step 3: Enhanced RAG Chatbot Class
# ============================================================================

class AuroraBayChatbot:
    """RAG Chatbot for Aurora Bay Town FAQs"""

    def __init__(self):
        try:
            self.model = GenerativeModel(GENERATIVE_MODEL)
            print("🤖 Aurora Bay Chatbot initialized successfully")
        except Exception as e:
            print(f"❌ Model initialization failed: {e}")
            self.model = None

    def build_context_prompt(self, user_question, relevant_faqs):
        """Build prompt with retrieved context"""

        if relevant_faqs.empty:
            return f"""
            CONTEXT: No specific Aurora Bay FAQ information found for this question.

            QUESTION: {user_question}

            Please provide a helpful response but clearly state that specific Aurora Bay information is not available.
            Always direct users to contact the Aurora Bay Town Hall for official information.
            """

        context = "OFFICIAL AURORA BAY FAQ INFORMATION:\n\n"
        for i, (_, faq) in enumerate(relevant_faqs.iterrows(), 1):
            similarity_score = faq['relevance_score']
            context += f"FAQ {i} (Relevance: {similarity_score:.1%}):\n"
            context += f"QUESTION: {faq['question']}\n"
            context += f"OFFICIAL ANSWER: {faq['answer']}\n\n"

        prompt = f"""
        ROLE: You are an official assistant for Aurora Bay, Alaska. Your responses must be accurate and based ONLY on the provided official information.

        OFFICIAL AURORA BAY FAQ CONTEXT:
        {context.rstrip()}

        USER QUESTION: {user_question}

        CRITICAL INSTRUCTIONS:
        1. Answer using ONLY the provided Aurora Bay FAQ context
        2. If the exact answer is in the context, provide it accurately and completely
        3. If the context has related information, synthesize it to answer the question
        4. If the context doesn't contain the specific information, say: "I don't have specific information about this in the Aurora Bay FAQs. Please contact the Town Hall at (907) 555-0123 for assistance."
        5. Never make up or hallucinate information
        6. Be specific about locations, procedures, and contact details when available
        7. Keep responses clear and helpful

        OFFICIAL RESPONSE:
        """

        return prompt

    def ask_question(self, user_question):
        """Main method to get answer using RAG"""

        print(f"\n🧑‍💼 USER: {user_question}")

        # Search for relevant FAQs
        relevant_faqs = semantic_search_fixed(user_question)

        # Build and execute prompt
        rag_prompt = self.build_context_prompt(user_question, relevant_faqs)

        print("🤖 Generating response with Gemini...")

        if self.model is None:
            return self._fallback_response(user_question, relevant_faqs)

        try:
            response = self.model.generate_content(rag_prompt)

            print(f"\n🏛️  AURORA BAY ASSISTANT:")
            print("=" * 60)
            print(response.text)
            print("=" * 60)

            return {
                "success": True,
                "response": response.text,
                "relevant_faqs_count": len(relevant_faqs),
                "method": "rag_with_gemini"
            }

        except Exception as e:
            print(f"❌ Gemini error: {e}")
            return self._fallback_response(user_question, relevant_faqs)

    def _fallback_response(self, user_question, relevant_faqs):
        """Provide fallback response when Gemini fails"""
        print(f"\n🏛️  AURORA BAY ASSISTANT (Direct FAQ Results):")
        print("=" * 60)
        if not relevant_faqs.empty:
            print("Based on Aurora Bay FAQs:")
            for i, (_, faq) in enumerate(relevant_faqs.iterrows(), 1):
                print(f"\n{i}. Q: {faq['question']}")
                print(f"   A: {faq['answer']}")
        else:
            print(f"I don't have specific information about '{user_question}' in the Aurora Bay FAQs.")
            print("\nFor official information, please contact:")
            print("📍 Aurora Bay Town Hall")
            print("📞 (907) 555-0123")
            print("📧 info@aurorabay.gov")
            print("🕐 Monday-Friday 8:00 AM - 5:00 PM")
        print("=" * 60)

        return {
            "success": False,
            "relevant_faqs_count": len(relevant_faqs),
            "method": "direct_faq"
        }

# ============================================================================
# Step 4: Test FAQ Retrieval with Correct Columns
# ============================================================================

def test_faq_retrieval_fixed():
    """Test if we can retrieve specific FAQs from the database with correct columns"""
    print("\n🧪 TESTING FAQ RETRIEVAL WITH CORRECT COLUMNS")
    print("=" * 60)

    test_queries = [
        "town hall",
        "business license",
        "power outage",
        "recycling",
        "council meetings",
        "building permit",
        "pet adoption",
        "water bill",
        "trash collection",
        "parks"
    ]

    for query in test_queries:
        print(f"\nSearching for: '{query}'")
        results = semantic_search_fixed(query, top_k=2)
        if not results.empty:
            for i, (_, row) in enumerate(results.iterrows(), 1):
                print(f"   {i}. {row['question'][:50]}... (score: {row['relevance_score']:.3f})")
        else:
            print("   No results found")

# ============================================================================
# Step 5: Explore Table Content
# ============================================================================

def explore_table_content():
    """Explore what's actually in the table"""
    print("\n🔍 EXPLORING TABLE CONTENT")
    print("=" * 60)

    try:
        # Get sample data to understand content
        explore_query = f"""
        SELECT
          string_field_0 as potential_question,
          string_field_1 as potential_answer,
          LENGTH(string_field_0) as question_length,
          LENGTH(string_field_1) as answer_length
        FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
        LIMIT 10
        """

        sample_data = client.query(explore_query).to_dataframe()
        print("Sample rows from the table:")
        for i, (_, row) in enumerate(sample_data.iterrows(), 1):
            print(f"\n{i}. QUESTION: {row['potential_question'][:80]}...")
            print(f"   ANSWER: {row['potential_answer'][:80]}...")

        return True
    except Exception as e:
        print(f"❌ Exploration failed: {e}")
        return False

# ============================================================================
# Step 6: Main Execution and Testing
# ============================================================================

def main():
    """Run complete RAG system testing"""

    print("🚀 AURORA BAY RAG SYSTEM - FIXED VERSION")
    print("=" * 60)

    # First, explore the table content
    explore_table_content()

    # Test FAQ retrieval with correct columns
    test_faq_retrieval_fixed()

    # Initialize chatbot
    print("\n🤖 Initializing RAG Chatbot...")
    chatbot = AuroraBayChatbot()

    # Comprehensive test questions
    test_questions = [
        "Where is the town hall located?",
        "What are the business license requirements?",
        "How do I report a power outage?",
        "What are the recycling guidelines?",
        "When are town council meetings held?",
        "How do I get a building permit?",
        "What are the pet adoption procedures?",
        "How do I pay my water bill?",
        "What is the procedure for trash collection?",
        "Where can I find information about local parks?"
    ]

    print(f"\n🧪 TESTING RAG SYSTEM WITH {len(test_questions)} QUESTIONS")
    print("=" * 60)

    successful_responses = 0
    found_faqs_count = 0

    for i, question in enumerate(test_questions, 1):
        print(f"\n📋 TEST {i}/10")
        result = chatbot.ask_question(question)

        if result and result.get('success', False):
            successful_responses += 1
        if result and result.get('relevant_faqs_count', 0) > 0:
            found_faqs_count += 1

    print("\n🎉 RAG SYSTEM TESTING COMPLETE!")
    print("=" * 60)
    print("📊 PERFORMANCE SUMMARY:")
    print(f"   ✅ Successful Gemini responses: {successful_responses}/10")
    print(f"   🔍 Questions with relevant FAQs: {found_faqs_count}/10")
    print(f"   🤖 Model: {GENERATIVE_MODEL}")
    print(f"   📍 Project: {PROJECT_ID}")
    print("=" * 60)

# ============================================================================
# Interactive Chat Session
# ============================================================================

def interactive_chat():
    """Run an interactive chat session"""
    print("\n" + "="*60)
    print("💬 INTERACTIVE AURORA BAY CHAT SESSION")
    print("="*60)
    print("Type 'quit' or 'exit' to end the session")
    print("Try questions like:")
    print("  - 'Where is the animal shelter?'")
    print("  - 'How do I pay utilities?'")
    print("  - 'What are the park hours?'")
    print("  - 'Business license requirements'")
    print("="*60)

    chatbot = AuroraBayChatbot()

    while True:
        user_input = input("\n🧑‍💼 You: ").strip()

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("🏛️  Thank you for visiting Aurora Bay!")
            break

        if not user_input:
            continue

        chatbot.ask_question(user_input)

# Run the complete system
if __name__ == "__main__":
    # Run automated tests first
    main()

    # Then start interactive session
    print("\n" + "="*60)
    print("🚀 Starting Interactive Chat Session...")
    print("="*60)
    interactive_chat()

🚀 Aurora Bay RAG System Initializing...
📍 Project: qwiklabs-gcp-00-cc0593714b16
📍 Location: us-central1
🤖 Generative Model: gemini-2.5-flash
🚀 AURORA BAY RAG SYSTEM - FIXED VERSION

🔍 EXPLORING TABLE CONTENT
Sample rows from the table:

1. QUESTION: When was Aurora Bay founded?...
   ANSWER: Aurora Bay was founded in 1901 by a group of fur traders who recognized the regi...

2. QUESTION: What is the population of Aurora Bay?...
   ANSWER: Aurora Bay has a population of approximately 3,200 residents, although it can fl...

3. QUESTION: Where is the Aurora Bay Town Hall located?...
   ANSWER: The Town Hall is located at 100 Harbor View Road, in the center of Aurora Bay, c...

4. QUESTION: Who is the current mayor of Aurora Bay?...
   ANSWER: The current mayor is Linda Greenwood, elected in 2021 for a four-year term....

5. QUESTION: What are the primary industries in Aurora Bay?...
   ANSWER: The primary industries include commercial fishing, tourism, and small-scale logg...

6. QUESTION